In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
import mysql.connector

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("CreditCardApprovalPipeline") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.executor.memory", "4g") \
    .config("spark.executor.cores", "2") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

In [4]:
!pip install mysql-connector

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 107.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mysql-connector: filename=mysql_connector-2.2.9-cp310-cp310-linux_x86_64.whl size=247950 sha256=383238a43eb05d9c1573c1742e74d500a4c2e7d4eb5a7bfcce1f969f23146fa2
  Stored in directory: /root/.cache/pip/wheels/76/48/9b/da67ff1a18fe8e9d428f9b1a177716d4a7d363d2bbe83bf6cf
Successfully built mysql-connector


In [1]:
gcs_bucket = "dataproc-staging-us-central1-458263062208-tw36mmqt"
gcs_data_path = f"gs://{gcs_bucket}/notebooks/jupyter/jupyter/Big Data Class Notebooks/Project/Project2/Data/application_record.csv"


In [2]:

application_df = spark.read.option("header", "true").option("inferSchema", "true").csv(gcs_data_path)


In [11]:
import mysql.connector
import pandas as pd

In [ ]:
!curl ifconfig.me # u need to add your public ip of the notebook used in gcp to the  gcp mysql instance
# else it can't connect to mysql instance

# (Mayank is using the gcp notebook here, not vscode)

34.67.156.136

In [12]:
mysql_config = {
    'host': '34.122.208.22',
    'user': 'root',
    'password': 'BigData@12345',
    'database': 'loan_data',
    'table_name': 'credit_record'
}

mysql_conn = mysql.connector.connect(
    host=mysql_config["host"],
    user=mysql_config["user"],
    password=mysql_config["password"],
    database=mysql_config["database"]
)


query = f"SELECT * FROM {mysql_config['table_name']}"
pandas_df = pd.read_sql(query, mysql_conn) # If you try to read via Spark JDBC, you will face issues. That is why using Pandas 
mysql_conn.close()

# IN the project as well, follow the same and you can have upto 5 mb file in SQL server

credit_df = spark.createDataFrame(pandas_df)

/opt/conda/miniconda3/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


### in production scenario, u can't use pandas to read the data as it will be large, 
### u will require spark jdbc , so refer the below link 

# https://sparkbyexamples.com/pyspark/pyspark-query-database-table-using-jdbc/

In [14]:
credit_df.show(5)

25/03/22 04:07:46 WARN TaskSetManager: Stage 3 contains a task of very large size (9759 KiB). The maximum recommended task size is 1000 KiB.


+-------+--------------+------+
|     ID|MONTHS_BALANCE|STATUS|
+-------+--------------+------+
|5001711|            -3|     0|
|5001711|            -2|     0|
|5001711|            -1|     0|
|5001711|             0|     X|
|5001712|           -18|     0|
+-------+--------------+------+
only showing top 5 rows



In [15]:
# Data Exploration
application_df.printSchema()
credit_df.printSchema()

application_df.show(5)
credit_df.show(5)

print(f"Application Records: {application_df.count()}")
print(f"Credit Records: {credit_df.count()}")

application_df.describe().show()
credit_df.describe().show()

root
 |-- ID: integer (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- FLAG_MOBIL: integer (nullable = true)
 |-- FLAG_WORK_PHONE: integer (nullable = true)
 |-- FLAG_PHONE: integer (nullable = true)
 |-- FLAG_EMAIL: integer (nullable = true)
 |-- OCCUPATION_TYPE: string (nullable = true)
 |-- CNT_FAM_MEMBERS: double (nullable = true)

root
 |-- ID: string (nullable = true)
 |-- MONTHS_BALANCE: long (nullable = true)
 |-- STATUS: string (nullable = true)

+-------+-----------+------------+---------------+--

25/03/22 04:09:03 WARN TaskSetManager: Stage 5 contains a task of very large size (9759 KiB). The maximum recommended task size is 1000 KiB.


+-------+--------------+------+
|     ID|MONTHS_BALANCE|STATUS|
+-------+--------------+------+
|5001711|            -3|     0|
|5001711|            -2|     0|
|5001711|            -1|     0|
|5001711|             0|     X|
|5001712|           -18|     0|
+-------+--------------+------+
only showing top 5 rows

Application Records: 438557


25/03/22 04:09:05 WARN TaskSetManager: Stage 9 contains a task of very large size (9759 KiB). The maximum recommended task size is 1000 KiB.


Credit Records: 1048575


+-------+-----------------+-----------+------------+---------------+-------------------+------------------+--------------------+--------------------+------------------+-----------------+-------------------+-----------------+----------+------------------+-------------------+-------------------+--------------------+-----------------+
|summary|               ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|       CNT_CHILDREN|  AMT_INCOME_TOTAL|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|         DAYS_BIRTH|    DAYS_EMPLOYED|FLAG_MOBIL|   FLAG_WORK_PHONE|         FLAG_PHONE|         FLAG_EMAIL|     OCCUPATION_TYPE|  CNT_FAM_MEMBERS|
+-------+-----------------+-----------+------------+---------------+-------------------+------------------+--------------------+--------------------+------------------+-----------------+-------------------+-----------------+----------+------------------+-------------------+-------------------+--------------------+-----------------

25/03/22 04:09:41 WARN TaskSetManager: Stage 15 contains a task of very large size (9759 KiB). The maximum recommended task size is 1000 KiB.


+-------+-----------------+-------------------+-------------------+
|summary|               ID|     MONTHS_BALANCE|             STATUS|
+-------+-----------------+-------------------+-------------------+
|  count|          1048575|            1048575|            1048575|
|   mean|5068286.424673486|-19.136998307226474|0.05824863961501482|
| stddev|46150.57850528666| 14.023497688326547| 0.3949878642052498|
|    min|          5001711|                -60|                  0|
|    max|          5150487|                  0|                  X|
+-------+-----------------+-------------------+-------------------+

